In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import count

In [14]:
spark = SparkSession.builder.appName("Customer Visits").getOrCreate()

# Define the data
data = [
    (1, 23),
    (2, 9),
    (4, 30),
    (5, 54),
    (6, 96),
    (7, 54),
    (8, 54)
]

# Define the schema
columns = ["visit_id", "customer_id"]

# Create DataFrame
visits_df = spark.createDataFrame(data, schema=columns)

# Show the DataFrame
print("Visits:")
visits_df.show()

# Define the data for transactions
transactions_data = [
    (2, 5, 310),
    (3, 5, 300),
    (9, 5, 200),
    (12, 1, 910),
    (13, 2, 970)
]

# Define the schema for transactions
transactions_columns = ["transaction_id", "visit_id", "amount"]

# Create DataFrame for transactions
transactions_df = spark.createDataFrame(transactions_data, schema=transactions_columns)

print("Transactions:")
transactions_df.show()


Visits:
+--------+-----------+
|visit_id|customer_id|
+--------+-----------+
|       1|         23|
|       2|          9|
|       4|         30|
|       5|         54|
|       6|         96|
|       7|         54|
|       8|         54|
+--------+-----------+

Transactions:
+--------------+--------+------+
|transaction_id|visit_id|amount|
+--------------+--------+------+
|             2|       5|   310|
|             3|       5|   300|
|             9|       5|   200|
|            12|       1|   910|
|            13|       2|   970|
+--------------+--------+------+



#Approach-1: Using Left Join

In [26]:
joined_df = visits_df.join(transactions_df, "visit_id", 'left' )
joined_df.show()

+--------+-----------+--------------+------+
|visit_id|customer_id|transaction_id|amount|
+--------+-----------+--------------+------+
|       1|         23|            12|   910|
|       2|          9|            13|   970|
|       4|         30|          NULL|  NULL|
|       7|         54|          NULL|  NULL|
|       6|         96|          NULL|  NULL|
|       5|         54|             9|   200|
|       5|         54|             3|   300|
|       5|         54|             2|   310|
|       8|         54|          NULL|  NULL|
+--------+-----------+--------------+------+



In [27]:
grouped_df = joined_df.filter(col("transaction_id" ).isNull()) \
                      .groupBy("customer_id") \
                      .agg(count("visit_id").alias("count_no_trans"))
grouped_df.show()

+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         54|             2|
|         96|             1|
|         30|             1|
+-----------+--------------+

